# Classifying trialwise CorrectGo and NoGo trials

There are a number of steps to this. Hopefully we can recycle previous code and be up fairly quickly!

1. Load beta data. Ideally this process should include a cache into a pure python object so we don't have to reload it each time.
2. Preprocess the data.
3. Do cross-validated training and testing. Ideally an inner loop to select best parameters, an outer loop to get cross-validated performance, and final training over all the data to get an image. The inner loop can be probably be handled within the package we use probably.

In [2]:
import socket
import yaml
hostname=socket.gethostname()
hostname='zzz'
with open('sst_config.yml', "r") as f:
    test_config= yaml.safe_load(f)#[hostname]

In [3]:
import sys
import os
import pandas as pd
import numpy as np



sys.path.append(os.path.abspath("../../ml/"))
from apply_loocv_and_save import load_and_preprocess
from dev_utils import read_yaml_for_host
import warnings


config_data = read_yaml_for_host("sst_config.yml")



python initialized for apply_loocv_and_save
cpus available; cpus to use:
10 9
10


In [4]:
import multiprocessing
import math
import nibabel as nib
import nilearn as nl
from nilearn.decoding import DecoderRegressor,Decoder
from sklearn.model_selection import KFold,GroupKFold,LeaveOneOut
cpus_available = multiprocessing.cpu_count()

cpus_to_use = min(cpus_available-1,math.floor(0.9*cpus_available))
print(cpus_to_use)

9


In [5]:
from dev_wtp_io_utils import cv_train_test_sets, asizeof_fmt
from nilearn.decoding import DecoderRegressor,Decoder

In [6]:
nonbids_data_path = config_data['nonbids_data_path']
ml_data_folderpath = nonbids_data_path + "fMRI/ml"


## Set up the paradigm

In [7]:

def trialtype_resp_trans_func(X):
    return(X.trial_type)


## Loading beta data

beta data is generally written in `load_multisubject_brain_data_sst_w1.ipynb`.

We just have to load it.

In [8]:
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_30subs_correct_cond_pfc.pkl'
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_6subs_correct_cond.pkl'
warnings.warn("not sure if this file holds up--it was created in 2021; need to see if it's still valid")
train_test_markers_filepath = ml_data_folderpath + "/train_test_markers_20220818T144138.csv"

/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_28423/2753699166.py:3: UserWarning: not sure if this file holds up--it was created in 2021; need to see if it's still valid
  warnings.warn("not sure if this file holds up--it was created in 2021; need to see if it's still valid")


In [9]:


all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = trialtype_resp_trans_func,
    clean=None)

warnings.warn("the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.")


checked for intersection and no intersection between the brain data and the subjects was found.
there were 6 subjects overlapping between the subjects marked for train data and the training dump file itself.


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


test_train_set: 62918
brain_data_filepath: 168
pkl_file: 168
train_test_markers_filepath: 158
response_transform_func: 144
sys: 72
Brain_Data_allsubs: 48
clean: 16
subjs_to_use: 16


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


601
601
cleaning memory


/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_28423/1474138623.py:8: UserWarning: the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.
  warnings.warn("the data hasn't been cleaned at any point. the fMRIPrep cleaning pipeline has been applied; nothing else has been.")


In [10]:
from sklearn.feature_selection import SelectPercentile,f_classif

#convert the y array to an integer array representing the string values of the y array
all_subjects['y_cat'] = all_subjects['y'].astype('category')
all_subjects['y_int']=all_subjects['y_cat'].cat.codes

# #get the X fdata
# x_fdata = all_subjects['X'].get_fdata()
# print(x_fdata.shape)
# #flatten first 3 dims of X fdata to a single dim
# x_fdata_1d = x_fdata.reshape([np.prod(x_fdata.shape[0:3]), x_fdata.shape[3]])
# print(x_fdata_1d.shape)



# #SelectPercentile(f_classif,percentile=5).fit_transform(,all_subjects['y_int'])

In [11]:
mask_nifti = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [12]:
num_subjs = 6
if num_subjs<len(all_subjects['metadata']['subject'].unique()):
    #cut down to just 9 subjects
    selected_subjs={}
    print(all_subjects.keys())

    #select subjs
    subjs = all_subjects['metadata']['subject'].unique()
    subjs.sort()
    selected_sub_ids=subjs[0:num_subjs]
    #get index of selected subjs
    selected_subjs_idx_bool = all_subjects['metadata']['subject'].isin(selected_sub_ids)
    #get list of True indices
    selected_subjs_idx = selected_subjs_idx_bool[selected_subjs_idx_bool].index.tolist()

    subjs = all_subjects['metadata']['subject'].unique()
    selected_subjs['X'] = nib.funcs.concat_images([all_subjects['X'].slicer[...,s] for s in selected_subjs_idx])
    selected_subjs['y'] = all_subjects['y'][selected_subjs_idx_bool]

    selected_subjs['y'] = all_subjects['y'][selected_subjs_idx_bool]
    selected_subjs['groups'] = all_subjects['groups'][selected_subjs_idx_bool]
    selected_subjs['metadata'] = all_subjects['metadata'][selected_subjs_idx_bool]
else:
    selected_subjs = all_subjects

In [13]:
# get the PFC mask
mask_nifti = nib.load(ml_data_folderpath + '/prefrontal_cortex.nii.gz')

In [14]:
#convert the y array to an integer array representing the string values of the y array
selected_subjs['y_cat'] = selected_subjs['y'].astype('category')
selected_subjs['y_int']=selected_subjs['y_cat'].cat.codes

In [32]:
X=selected_subjs['X']
myarr=X.get_fdata()
#rotate my array so that the last dimension is first
print(myarr.shape)
myarr=np.moveaxis(myarr,-1,0)
print(myarr.shape)
pre_reshaped_shape = myarr.shape
#now flatten dims 2-4 into a single dimension
myarr2d=myarr.reshape([myarr.shape[0],np.prod(myarr.shape[1:4])])
print("this is the form of the data that the decoder wants, (n_samples, n_features)")
print(myarr2d.shape)


# print("stats on the ")
# #check the means of each of the matrices
# #just hte first 10 perhaps
# myarr_2d_
# print(len())
# print(myarr.mean(axis=1)[0:10])
#now rotate back
myarr2d=np.moveaxis(myarr2d,0,-1)
print(myarr2d.shape)

#now undo the above operations to get the original matrix
myarr_rev=np.moveaxis(myarr2d,-1,0)
print(myarr_rev.shape)
myarr_rev=myarr_rev.reshape(pre_reshaped_shape)
print(myarr_rev.shape)
myarr_rev=np.moveaxis(myarr,0,-1)
print(myarr_rev.shape)
print("finally test they're equivalent.")
myarr_orig=X.get_fdata()

print(np.sum(myarr_orig==myarr_rev))

print(np.prod(myarr_rev.shape))


(97, 115, 97, 601)
(601, 97, 115, 97)
this is the form of the data that the decoder wants, (n_samples, n_features)
(601, 1082035)
(1082035, 601)
(601, 1082035)
(601, 97, 115, 97)
(97, 115, 97, 601)
finally test they're equivalent.
650303035
650303035


In [31]:
#and now test that in the consolidated array, the means for each image are the same as they are in the oriignal
arr2d_means = myarr2d.mean(axis=0)
print(len(arr2d_means))
arrorig_means = myarr_orig.mean(axis=(0,1,2))
print(len(arrorig_means))

print(np.all((np.round(arr2d_means,3)==np.round(arrorig_means,3))))


601
601
True


Let's write a function that transforms an array into 2D n_samples, n_features,
and another function that transforms it back


In [33]:
def get_2DX_from_4DX(nifti_X):
    #rotate my array so that the last dimension is first
    print(nifti_X.shape)
    nifti_X=np.moveaxis(nifti_X,-1,0)
    print(nifti_X.shape)
    pre_reshaped_shape = nifti_X.shape
    #now flatten dims 2-4 into a single dimension
    arr2d=myarr.reshape([nifti_X.shape[0],np.prod(nifti_X.shape[1:4])])
    print("this is the form of the data that the decoder wants, (n_samples, n_features)")
    print(arr2d.shape)
    return(arr2d)


def get_4DX_from_2DX(arr2d,pre_reshaped_shape):
    #now undo the above operations to get the original matrix
    print(arr2d.shape)
    arr4d=arr2d.reshape(pre_reshaped_shape)
    print(myarr_rev.shape)
    arr4d=np.moveaxis(arr4d,0,-1)
    print(arr4d.shape)
    return(arr4d)